### Import packages

In [1]:
%cd ../

/work/yuxiang1234/sandbox-AND


In [2]:
import os
import json
import sys
from collections import defaultdict

import clip
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
# from vllm import LLM, SamplingParams

import similarity
from sentence_utils import (get_basename, post_process_prediction)
from data_utils import (save_discriminative_sample, get_clip_prediction, get_cls_id_to_label,
                        get_concept_set, get_topk_acc, get_concept_dataset,
get_audio_description, get_similarity_from_descriptions, read_json)
from utils import save_activations, get_similarity_from_activations


/home/yuxiang1234/env/dissect/lib/python3.8/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/yuxiang1234/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Arguments

In [3]:
class Arguments:
    def __init__(self):

        self.target_name = "ast-esc50" # Model to dissect (target model)
        
        self.target_layers = "fc"
        # self.target_layers = "layer0_output,layer0_intermediate,layer0_attention_output, \
        #                 	  layer1_output,layer1_intermediate,layer1_attention_output, \
        #                       layer2_output,layer2_intermediate,layer2_attention_output, \
        #                       layer3_output,layer3_intermediate,layer3_attention_output, \
        #                       layer4_output,layer4_intermediate,layer4_attention_output, \
        #                       layer5_output,layer5_intermediate,layer5_attention_output, \
        #                       layer6_output,layer6_intermediate,layer6_attention_output, \
        #                       layer7_output,layer7_intermediate,layer7_attention_output, \
        #                       layer8_output,layer8_intermediate,layer8_attention_output, \
        #                       layer9_output,layer9_intermediate,layer9_attention_output, \
        #                       layer10_output,layer10_intermediate,layer10_attention_output, \
        #                       layer11_output,layer11_intermediate,layer11_attention_output,fc"
        # Which layer neurons to describe. String list of layer names to describe, separated by comma (no spaces). 
        # Follows the naming format of the Pytorch module used.

        # self.target_layers = "layer0_1,layer0_2,layer1_1,layer1_2,layer2_1,layer2_2,layer3_1,layer3_2,layer4_1,layer4_2,layer5_1,layer5_2,layer6_1,layer6_2,layer7_1,layer7_2,layer8_1,layer8_2,layer9_1,layer9_2,layer10_1,layer10_2,layer11_1,layer11_2,fc"
        
        self.probing_dataset = "esc50"  # Probing dataset to probe the target model
        self.concept_set_file = "data/concept_set/esc50.txt"  # Path to txt file of concept set
        self.network_class_file = "data/network_class/esc50.txt"  # Path to txt file of network's classification class
        self.clip_model = "ViT-B/32"  # CLIP model version to use
        self.clap_model = "ViT-B/32"  # CLAP model version to use
        self.sentence_transformer = 'all-MiniLM-L12-v2'  # Sentence transformer to use
        self.batch_size = 1  # Batch size when running CLIP/target model
        self.device = "cuda"  # Whether to use GPU/which GPU
        self.seed = 20  # Seed number
        self.num_of_gpus = 1  # Number of available GPUs for vllm
        self.pool_mode = "avg"  # Aggregation function for channels
        self.scoring_func = False  # Scoring function flag

        # Directory paths
        self.audio_description_dir = "audio_description"  # Directory to save audio descriptions
        self.audio_dir = "save_audios"  # Directory to save audio
        self.save_activation_dir = "saved_activations"  # Directory to save activation values
        self.save_summary_dir = "summaries"  # Directory to save summaries
        self.save_discriminative_sample_dir = "discriminative_samples"  # Directory to save discriminative samples
        self.save_prediction_dir = "prediction"  # Directory to save prediction
        self.save_interpretability_dir = 'interpretability'  # Directory to save interpretability experiments

        # Discriminative settings
        self.discriminative_type = "highly"  # Type of discriminative samples
        self.post_process_type = "sim"  # Post-processing type
        self.mutual_info_threshold = 0.6  # Mutual information threshold
        self.K = 5  # Top-K highly/lowly-activated audio
        self.clusters = 11  # Number of clusters

        # LLM settings
        self.llm = "meta-llama/Llama-2-13b-chat-hf"  # LLM to use
        self.top_p = 1.0  # Sampling parameter: top-p
        self.temperature = 1.0  # Sampling parameter: temperature
        self.max_tokens = 128  # Sampling parameter: max tokens
        self.ICL_topk = 1  # Experiments of top5 or top1 accuracy of ICL

        # Pruning settings
        self.save_pruning_dir = "pruning_result"  # Directory to save pruning results
        self.max_pruned_num = 3000  # Maximum number of pruned neurons
        self.pruned_concepts = ["water_drops"]  # Concepts to be ablated
        self.pruning_strategy = "db"  # Method to decide pruned neurons (random, db, tab, ocp)

In [5]:
args = Arguments()

### Preprocess

In [6]:
args.save_activation_dir = os.path.join(args.save_activation_dir, f"{args.target_name}_{args.probing_dataset}_{get_basename(args.concept_set_file)}")

save_activations(args)

args.target_layers = args.target_layers.split(",")

dataset = save_discriminative_sample(args.save_discriminative_sample_dir, args.save_activation_dir, args.probing_dataset, args.concept_set_file, args.target_name, args.target_layers, args.K)


/home/yuxiang1234/env/dissect/lib/python3.8/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/yuxiang1234/env/dissect/lib/python3.8/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm2.we

  0%|          | 0/1 [00:00<?, ?it/s]

file exist, load saved_activations/ast-esc50_esc50_esc50/text_esc50.pt
file exist, load saved_activations/ast-esc50_esc50_esc50/text_esc50.pt


  0%|          | 0/1 [00:00<?, ?it/s]

target layer: fc


### Closed-concept Identification

#### DB

In [7]:
clip_model, clip_preprocess = clip.load(args.clip_model, device=args.device)
transformer_model = SentenceTransformer(args.sentence_transformer)

concepts = get_concept_set(args.concept_set_file)
descriptions = get_audio_description(args.audio_description_dir, args.probing_dataset)

cls_id_to_label = get_cls_id_to_label(args.network_class_file)

discriminative_sample_file = os.path.join(args.save_discriminative_sample_dir, f"{args.target_name}_{args.probing_dataset}_{get_basename(args.concept_set_file)}.json")
neuron_ordered_activation = read_json(discriminative_sample_file)

# similarity_names = ["cos_similarity_cubed"]
# similarity_fns = [similarity.cos_similarity_cubed]
similarity_names = ["cos-similarity", "cos_similarity_cubed", "rank_reorder", "wpmi", "soft_wpmi"]
similarity_fns = [similarity.cos_similarity, similarity.cos_similarity_cubed, similarity.rank_reorder, similarity.wpmi, similarity.soft_wpmi]

target_layers = args.target_layers
results = defaultdict(lambda: defaultdict(dict))
args.save_dir = os.path.join(args.save_activation_dir, f'{args.target_name}_{args.probing_dataset}_{get_basename(args.concept_set_file)}')

with open(args.concept_set_file, 'r') as f: 
	concept_features = []
	bs = 100
	for step in range(0, len(concepts) // bs + 1):
		con = concepts[bs*step: bs*(step+1)]
		word = clip.tokenize(con, truncate=True).to(args.device)
		word_feature = clip_model.encode_text(word).detach().cpu()
		concept_features.append(word_feature)
	concept_features = torch.cat(concept_features, 0).to(args.device).float()

for target_layer in target_layers:
	for similarity_fn, similarity_name in zip(similarity_fns, similarity_names):
		
		similarities, keys = get_similarity_from_descriptions(target_layer, clip_model, 
			neuron_ordered_activation, descriptions, concept_features, similarity_fn, device=args.device)

		print('similarity function: ',similarity_name)
		if target_layer == 'fc':
			num_of_neuron = len([value for value in cls_id_to_label.values() if value != None])
			print(f"Calculate accuracy on {num_of_neuron} neurons out of {len(cls_id_to_label)} neurons")

			print(f"{similarity_name} Top-1 fc acc:{get_topk_acc(similarities, cls_id_to_label, concepts, k=1):.4f}")
			print(f"{similarity_name} Top-5 fc acc:{get_topk_acc(similarities, cls_id_to_label, concepts, k=5):.4f}")

			pred = [concepts[int(p)] for p in torch.argmax(similarities, dim=1)]

			gt_embeds = transformer_model.encode([cls_id_to_label[i] for i in range(len(cls_id_to_label))])
			pred_embeds = transformer_model.encode(pred)
			cos_sim_mpnet = np.sum(pred_embeds * gt_embeds, axis=1)

			cos_sim = float(np.mean(cos_sim_mpnet))
			print("DB {} fc cos sim: {:.4f}".format(similarity_name, cos_sim))

		prediction, gt = get_clip_prediction(similarities, cls_id_to_label, concepts, K=args.K, final_layer=(target_layer=='fc'))

		for i in range(len(similarities)):
			results[keys[i]][similarity_name] = {'gt': gt[i], f'prediction': prediction[i]} if target_layer == 'fc' else {'gt': None, 'prediction': prediction[i]}

if not os.path.exists(args.save_prediction_dir):
	os.makedirs(args.save_prediction_dir)

with open(os.path.join(args.save_prediction_dir, f"db-{args.target_name}-top{args.K}.json"), "w") as f: 
	json.dump(results, f, indent=2)


similarity function:  cos-similarity
Calculate accuracy on 50 neurons out of 50 neurons
cos-similarity Top-1 fc acc:80.0000
cos-similarity Top-5 fc acc:82.0000
DB cos-similarity fc cos sim: 0.8487
similarity function:  cos_similarity_cubed
Calculate accuracy on 50 neurons out of 50 neurons
cos_similarity_cubed Top-1 fc acc:100.0000
cos_similarity_cubed Top-5 fc acc:100.0000
DB cos_similarity_cubed fc cos sim: 1.0000
similarity function:  rank_reorder
Calculate accuracy on 50 neurons out of 50 neurons
rank_reorder Top-1 fc acc:86.0000
rank_reorder Top-5 fc acc:100.0000
DB rank_reorder fc cos sim: 0.9331
similarity function:  wpmi
Calculate accuracy on 50 neurons out of 50 neurons
wpmi Top-1 fc acc:88.0000
wpmi Top-5 fc acc:100.0000
DB wpmi fc cos sim: 0.9466
similarity function:  soft_wpmi
Calculate accuracy on 50 neurons out of 50 neurons
soft_wpmi Top-1 fc acc:2.0000
soft_wpmi Top-5 fc acc:10.0000
DB soft_wpmi fc cos sim: 0.2356


#### ICL

In [ ]:
if args.ICL_topk == 1:
	prompt_template =  "<s>[INST] <<SYS>>\n\
	You have a set of object classnames: \
	{} \
	\n \
	The following is a description about some audio clips. Based on the description, select a classname out of the above classnames that matches the description most. \
	<</SYS>>\n\n \
	The audio features a car meowing: All of the clips contain the sound of a cat meowing. Loud sound: These clips are all of loud sound but with varying degrees of intensity. Repetitive barking: Clips 1 and 4 are repetitive, with the cat meowing multiple times in each clip. Poor audio quality: All clips have poor audio quality, with either distortion, muffling, or apparent background noises. \
	[/INST] \
	We know these clips are about the class 'cat' in the concept set. We can get this answer since the description mentions All of the clips contain the sound of a cat meowing'. \n \
	Answer: cat \
	</s><s>[INST] \
	They all feature a person snoring loudly. The snoring starts off slow and gets louder over time. The audio is recorded in mono. There are no other sounds in the background. The snoring is described as loud and intense. The audio clips differ in the following ways. The first clip features a man snoring, while the second and fourth clips feature a person snoring (gender not specified). The third clip features a zombie growling and snarling, while the other clips only feature snoring. The third clip is described as scary and creepy, while the other clips are not. The third clip is intended for use in a horror movie or zombie video game, while the other clips do not have specific intended uses stated. The third clip is of poor quality, while the other clips are not specified as such. \
	[/INST] \
	Based on the description, the most suitable classname for the audio clips would be 'snoring' or 'zombie_growling_and_snarling'. Both of these classnames match the description of loud sounds with a strong emotional impact, specifically fear and terror. But 'zombie_growling_and_snarling' is not in the given classname set. So the answer is 'snoring' \
	Answer: soring \
	</s><s>[INST] \
	{} \
	[/INST]"
else:
	prompt_template =  "<s>[INST] <<SYS>>\n\
	You have a set of object classnames: \
	{} \
	\n \
	The following is a description about some audio clips. Based on the description, select 5 classnames out of the above classnames that matches the description most and given the reason. There should be exactly 5 answers.  \
	<</SYS>>\n\n \
	The audio features a cat meowing: All of the clips contain the sound of a cat meowing. Loud sound: These clips are all of loud sound but with varying degrees of intensity. Repetitive barking: Clips 1 and 4 are repetitive, with the cat meowing multiple times in each clip. Poor audio quality: All clips have poor audio quality, with either distortion, muffling, or apparent background noises. \
	[/INST] \
	We know these clips are about the class 'cat' in the concept set. We can get this answer since the description mentions All of the clips contain the sound of a cat meowing'. \n \
	Answer: cat, sheep, snoring, cow, coughing \
	</s><s>[INST] \
	They all feature a person snoring loudly. The snoring starts off slow and gets louder over time. The audio is recorded in mono. There are no other sounds in the background. The snoring is described as loud and intense. The audio clips differ in the following ways. The first clip features a man snoring, while the second and fourth clips feature a person snoring (gender not specified). The third clip features a zombie growling and snarling, while the other clips only feature snoring. The third clip is described as scary and creepy, while the other clips are not. The third clip is intended for use in a horror movie or zombie video game, while the other clips do not have specific intended uses stated. The third clip is of poor quality, while the other clips are not specified as such. \
	[/INST] \
	Based on the description, the most suitable classname for the audio clips would be 'snoring' or 'zombie_growling_and_snarling'. Both of these classnames match the description of loud sounds with a strong emotional impact, specifically fear and terror. But 'zombie_growling_and_snarling' is not in the given classname set. So the answer is 'snoring' \
	Answer: snoring, zombie_growling_and_snarling, breathing, footsteps, coughing\
	</s><s>[INST] \
	{} \
	[/INST]"

dataset = get_concept_dataset(args.save_summary_dir, args.probing_dataset, args.concept_set_file, args.target_name, args.target_layers, args.network_class_file, prompt_template, K=args.K)

llm = LLM(model=args.llm, gpu_memory_utilization=0.9)
sampling_params = SamplingParams(top_p=args.top_p, temperature=args.temperature, max_tokens=512)

results = []
inputs = [line["text"] for line in dataset]
outputs = llm.generate(inputs, sampling_params)

for inp, out in zip(dataset, outputs):
	target_layer = inp["target_layer"]
	neuron_id = inp["neuron_id"]
	neuron_label = inp["neuron_label"]
	print(f'-------- {target_layer} # {neuron_id}  --------')

	response = out.outputs[0].text
	results.append({"response": response,
					"target_layer": target_layer,
					"neuron_id": neuron_id,
					"neuron_label": neuron_label})

	if target_layer == "fc":
		print("gt label: ", neuron_label)
		print(response)

	print("=" * 20)

# calculate accuracies
concepts = get_concept_set(args.concept_set_file)

if not os.path.exists(args.save_prediction_dir):
	os.makedirs(args.save_prediction_dir)

is_correct, prediction, cos_sim = [], [], []
for line in results:
	prediction, sim = post_process_prediction(line["response"], concepts=concepts, gt=line["neuron_label"], embedding_model=args.sentence_transformer, K=args.ICL_topk)
	
	is_correct.append(1) if line["neuron_label"] in prediction else is_correct.append(0)
	cos_sim.append(sim)

print("accuracy: ", round(sum(is_correct) / len(is_correct) * 100, 2))
print("cos sim : ", round(sum(cos_sim) / len(cos_sim), 2))

# save prediction
prediction_file = os.path.join(args.save_prediction_dir, f'{args.target_name}_{args.probing_dataset}_{args.concept_set_file.split("/")[-1].split(".txt")[0]}_{args.K}_{args.ICL_topk}.json')

with open(prediction_file, "w") as f:
	json.dump(results, f, indent=2)

#### TAB

In [10]:
transformer_model = SentenceTransformer(args.sentence_transformer)

concepts = get_concept_set(args.concept_set_file)
cls_id_to_label = get_cls_id_to_label(args.network_class_file)

similarity_names = ["cos-similarity", "cos_similarity_cubed", "rank_reorder", "wpmi", "soft_wpmi"]
similarity_fns = [similarity.cos_similarity, similarity.cos_similarity_cubed, similarity.rank_reorder, similarity.wpmi, similarity.soft_wpmi]

results = defaultdict(lambda: defaultdict(dict))

for target_layer in target_layers:
	target_save_name = f"{args.save_activation_dir}/target_{args.probing_dataset}_{args.target_name}_{target_layer}.pt"
	audio_save_name = f"{args.save_activation_dir}/audio_{args.probing_dataset}.pt"
	text_save_name = f"{args.save_activation_dir}/text_{args.concept_set_file.split('/')[-1].replace('.txt', '')}.pt"

	for similarity_fn, similarity_name in zip(similarity_fns, similarity_names):

		similarities, target_feats = get_similarity_from_activations(target_save_name,   
			audio_save_name, text_save_name, similarity_fn, device=args.device)
		
		if target_layer == "fc":
			print(similarity_name)
			num_of_neuron = len([value for value in cls_id_to_label.values() if value != None])
			print(f"Calculate accuracy on {num_of_neuron} neurons out of {len(cls_id_to_label)} neurons")
			concepts = get_concept_set(args.concept_set_file, clip_format=False)

			print(f"TAB Top 1 acc:{get_topk_acc(similarities, cls_id_to_label, concepts, k=1):.4f}")
			print(f"TAB Top 5 acc:{get_topk_acc(similarities, cls_id_to_label, concepts, k=5):.4f}")


			prediction, gt = get_clip_prediction(similarities, cls_id_to_label, concepts, K=args.K, final_layer=True)

			# pred = torch.argmax(similarities, dim=1)
			# pred = [concepts[int(p)] for p in pred]
			
			gt_embeds = transformer_model.encode([cls_id_to_label[i] for i in range(len(cls_id_to_label))])
			pred_embeds = transformer_model.encode(prediction)
			cos_sim_mpnet = np.sum(pred_embeds * gt_embeds, axis=1)

			cos_sim = float(np.mean(cos_sim_mpnet))
			print("TAB cos sim   :{:.4f}".format(cos_sim))

			for i in range(len(similarities)):
				key = f"{target_layer}#{i}"
				results[key][similarity_name] = {'gt': gt[i], f'prediction': prediction[i]}
		else:
			prediction, _ = get_clip_prediction(similarities, cls_id_to_label, concepts, K=args.K, final_layer=False)

			for i in range(len(similarities)):
				key = f"{target_layer}#{i}"
				results[key][similarity_name] = {'gt': None, f'prediction': prediction[i]}

if os.path.exists(args.save_prediction_dir) == False:
	os.makedirs(args.save_prediction_dir)

with open(os.path.join(args.save_prediction_dir, f"tab-{args.target_name}-top{args.K}.json"), "w") as f: 
	json.dump(results, f, indent=2)

cos-similarity
Calculate accuracy on 50 neurons out of 50 neurons
TAB Top 1 acc:2.0000
TAB Top 5 acc:16.0000
TAB cos sim   :0.2855
cos_similarity_cubed
Calculate accuracy on 50 neurons out of 50 neurons
TAB Top 1 acc:92.0000
TAB Top 5 acc:98.0000
TAB cos sim   :0.7838
rank_reorder
Calculate accuracy on 50 neurons out of 50 neurons
TAB Top 1 acc:84.0000
TAB Top 5 acc:92.0000
TAB cos sim   :0.7789
wpmi
Calculate accuracy on 50 neurons out of 50 neurons
TAB Top 1 acc:96.0000
TAB Top 5 acc:100.0000
TAB cos sim   :0.7932
soft_wpmi
Calculate accuracy on 50 neurons out of 50 neurons
TAB Top 1 acc:96.0000
TAB Top 5 acc:98.0000
TAB cos sim   :0.7343
